# Fetching Pre and Post synaptic Neuron Location

## Libraries

In [1]:
# ! pip install standard-transform
# ! pip install --upgrade caveclient

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for standard-transform: filename=standard_transform-1.4.0-py3-none-any.whl size=22356 sha256=7d71ebdb07043257fe88bbfb72e82f7bc2855579a1af5eae5b4f3cefa9b047c8
  Stored in directory: /home/vscode/.cache/pip/wheels/ad/3f/3f/2d6f4450cc3c31eb60e2dc6c4a24648a2e9563b610b26c9b67
Successfully built standard-transform

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for caveclient: filename=caveclient-5.13.1-py3-none-any.whl size=73839 sha256=d5e25dcc86b4d90ad694aeefa1e896899de53f33b4b59f494050c07af08a1355
  Stored in directory: /home/vscode/.cache/pip/wheels/43/fe/e9/cb5337e8fc6c865378718

In [1]:
import numpy as np
import pandas as pd
from caveclient import CAVEclient
from microns_phase3 import nda
from standard_transform import minnie_transform_vx

Connecting microns@db.datajoint.com:3306


Could not create the ~log table
Could not access table `microns_phase3_nda`.`~log`


## Preparation

Function that transforms the voxel coorindate into a depth coordinate (function given to us by the authors of the Microns Project)

In [2]:
t_form_v = minnie_transform_vx()

### Querying the data from CAVEclient

In [3]:
client = CAVEclient()

In [4]:
client.auth.save_token("64ea0271652fd86dc7fb6376cd9d52b9")

We will be using the latest public release of the dataset, i.e. version 661

In [27]:
client = CAVEclient('minnie65_public')

In [28]:
client.materialize.version = 661

In [29]:
client.materialize.get_tables()

['synapses_pni_2',
 'baylor_gnn_cell_type_fine_model_v2',
 'nucleus_alternative_points',
 'connectivity_groups_v507',
 'proofreading_status_public_release',
 'allen_column_mtypes_v1',
 'allen_v1_column_types_slanted_ref',
 'aibs_column_nonneuronal_ref',
 'nucleus_ref_neuron_svm',
 'aibs_soma_nuc_exc_mtype_preds_v117',
 'baylor_log_reg_cell_type_coarse_v1',
 'apl_functional_coreg_forward_v5',
 'nucleus_detection_v0',
 'aibs_soma_nuc_metamodel_preds_v117',
 'coregistration_manual_v3']

These are all the available tables, but we are interested in the *coregistration_manual_v3*, that is the one containing the information about the manually proofread neurons and in the *synapses_pni_2*, which contains all the traced synapses.

In [41]:
df = client.materialize.query_table('coregistration_manual_v3')

,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,pt_root_id,pt_position,bb_start_position,bb_end_position
0,13434,2023-04-05 22:39:53.474202+00:00,t,582276,7,5,4909,4,6.10722,9.568690,582276,2020-09-28 22:41:42.536121+00:00,t,274.291589,111332012547320963,864691135738685297,"[338720, 125232, 19589]","[nan, nan, nan]","[nan, nan, nan]"
1,3045,2023-04-05 22:39:03.110911+00:00,t,194144,7,4,9575,6,8.55708,-0.714900,194144,2020-09-28 22:42:01.511773+00:00,t,213.307228,83542405709639148,864691135614842827,"[136400, 170640, 17951]","[nan, nan, nan]","[nan, nan, nan]"
2,1474,2023-04-05 22:36:19.477499+00:00,t,293318,9,4,8066,6,5.63758,8.915025,293318,2020-09-28 22:45:23.572432+00:00,t,430.108180,90783515419343652,864691135502985397,"[189472, 119296, 26037]","[nan, nan, nan]","[nan, nan, nan]"
3,7915,2023-04-05 22:39:15.451639+00:00,t,332833,6,7,6413,6,5.27806,7.808064,332833,2020-09-28 22:44:41.864456+00:00,t,274.418729,93535317407139914,864691135387371905,"[209328, 174304, 20004]","[nan, nan, nan]","[nan, nan, nan]"
4,11589,2023-04-05 22:39:47.420305+00:00,t,295460,9,3,7748,6,5.20837,2.999543,295460,2020-09-28 22:41:24.368712+00:00,t,208.616653,90225718084406516,864691135661410544,"[185088, 157776, 15691]","[nan, nan, nan]","[nan, nan, nan]"


We start by setting as the index he ID associated to the root of the synapse

In [42]:
df = df.set_index('pt_root_id')

In [43]:
df.head()

,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,pt_position,bb_start_position,bb_end_position
pt_root_id,,,,,,,,,,,,,,,,,,
864691135738685297,13434,2023-04-05 22:39:53.474202+00:00,t,582276,7,5,4909,4,6.10722,9.568690,582276,2020-09-28 22:41:42.536121+00:00,t,274.291589,111332012547320963,"[338720, 125232, 19589]","[nan, nan, nan]","[nan, nan, nan]"
864691135614842827,3045,2023-04-05 22:39:03.110911+00:00,t,194144,7,4,9575,6,8.55708,-0.714900,194144,2020-09-28 22:42:01.511773+00:00,t,213.307228,83542405709639148,"[136400, 170640, 17951]","[nan, nan, nan]","[nan, nan, nan]"
864691135502985397,1474,2023-04-05 22:36:19.477499+00:00,t,293318,9,4,8066,6,5.63758,8.915025,293318,2020-09-28 22:45:23.572432+00:00,t,430.108180,90783515419343652,"[189472, 119296, 26037]","[nan, nan, nan]","[nan, nan, nan]"
864691135387371905,7915,2023-04-05 22:39:15.451639+00:00,t,332833,6,7,6413,6,5.27806,7.808064,332833,2020-09-28 22:44:41.864456+00:00,t,274.418729,93535317407139914,"[209328, 174304, 20004]","[nan, nan, nan]","[nan, nan, nan]"
864691135661410544,11589,2023-04-05 22:39:47.420305+00:00,t,295460,9,3,7748,6,5.20837,2.999543,295460,2020-09-28 22:41:24.368712+00:00,t,208.616653,90225718084406516,"[185088, 157776, 15691]","[nan, nan, nan]","[nan, nan, nan]"


In [44]:
df.shape

(13925, 18)

In [46]:
unit_keys_1 = df.iloc[:5000, :][['session', 'scan_idx', 'unit_id']].to_dict(orient='records')
unit_keys_2 = df.iloc[5000:10000, :][['session', 'scan_idx', 'unit_id']].to_dict(orient='records')
unit_keys_3 = df.iloc[10000:13925, :][['session', 'scan_idx', 'unit_id']].to_dict(orient='records')

In [51]:
temp_1 = (nda.AreaMembership & unit_keys_1).fetch('brain_area')
print('1/3 Completed!')
temp_2 = (nda.AreaMembership & unit_keys_2).fetch('brain_area')
print('2/3 Completed!')
temp_3 = (nda.AreaMembership & unit_keys_3).fetch('brain_area')
print('3/3 Completed!')
area_membership = np.concatenate((temp_1, temp_2, temp_3), axis=0)
assert len(area_membership) == df.shape[0]

/usr/local/lib/python3.9/site-packages/datajoint/connection.py:219: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


1/3 Completed!
2/3 Completed!
3/3 Completed!


In [52]:
df['Area Membership'] = area_membership
df.head()

,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,pt_position,bb_start_position,bb_end_position,Area Membership
pt_root_id,,,,,,,,,,,,,,,,,,,
864691135738685297,13434,2023-04-05 22:39:53.474202+00:00,t,582276,7,5,4909,4,6.10722,9.568690,582276,2020-09-28 22:41:42.536121+00:00,t,274.291589,111332012547320963,"[338720, 125232, 19589]","[nan, nan, nan]","[nan, nan, nan]",V1
864691135614842827,3045,2023-04-05 22:39:03.110911+00:00,t,194144,7,4,9575,6,8.55708,-0.714900,194144,2020-09-28 22:42:01.511773+00:00,t,213.307228,83542405709639148,"[136400, 170640, 17951]","[nan, nan, nan]","[nan, nan, nan]",V1
864691135502985397,1474,2023-04-05 22:36:19.477499+00:00,t,293318,9,4,8066,6,5.63758,8.915025,293318,2020-09-28 22:45:23.572432+00:00,t,430.108180,90783515419343652,"[189472, 119296, 26037]","[nan, nan, nan]","[nan, nan, nan]",V1
864691135387371905,7915,2023-04-05 22:39:15.451639+00:00,t,332833,6,7,6413,6,5.27806,7.808064,332833,2020-09-28 22:44:41.864456+00:00,t,274.418729,93535317407139914,"[209328, 174304, 20004]","[nan, nan, nan]","[nan, nan, nan]",V1
864691135661410544,11589,2023-04-05 22:39:47.420305+00:00,t,295460,9,3,7748,6,5.20837,2.999543,295460,2020-09-28 22:41:24.368712+00:00,t,208.616653,90225718084406516,"[185088, 157776, 15691]","[nan, nan, nan]","[nan, nan, nan]",V1


In [55]:
root_ids = df.index.tolist()
len(root_ids)

[864691135738685297,
 864691135614842827,
 864691135502985397,
 864691135387371905,
 864691135661410544,
 864691135349275351,
 864691137197706305,
 864691137197706305,
 864691136602371025,
 864691136602371025,
 864691135065322052,
 864691135731072697,
 864691135565275479,
 864691136194984268,
 864691135938356484,
 864691135975395439,
 864691136923754212,
 864691136923754212,
 864691136619416973,
 864691136673575047,
 864691135544785448,
 864691135272867601,
 864691135272867601,
 864691135272867601,
 864691135577813765,
 864691136309871706,
 864691135737772676,
 864691135866987798,
 864691135759371086,
 864691135013895574,
 864691136145003159,
 864691135407816393,
 864691136119541016,
 864691135463695429,
 864691135841932515,
 864691135725228203,
 864691135133547168,
 864691137020483694,
 864691136379765205,
 864691135233954137,
 864691136324904087,
 864691135195405354,
 864691135494448016,
 864691136085179244,
 864691136085179244,
 864691135292343990,
 864691136195216460,
 864691135416

In [56]:
input_dfs = [client.materialize.synapse_query(post_ids=i) for i in root_ids[]]